The goal of this notebook is to train a random_forest classifier on our cleaned dataset.

Here is a link to a data dictionary (which explains what the column names mean): https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/user-object.

See notebooks 4, 5, and 6 from the Classification lectures.

- [x] Implement a basic train-test split
- [x] Sample the data for training
- [x] Train one model
- [x] Quantify the model's accuracy
- [x] Monkey with hyperparameters, etc.
- [x] Make a more sophisticated train-test split
- [ ] Implement K-fold cross validation
- [ ] Create a proper scikit-learn pipeline
- [ ] Make some decent visualizations
- [ ] Add the categorical variables to dataframe (starting by making binary categories into boolean variables)
- [ ] One-hot encode the categorical variables, if necessary
- [ ] Examine feature importance
- [ ] Examine AUC

## Front matter

In [2]:
from pathlib import Path # For navigating to the datafiles
import json # For processing json files
import csv # For processing csv files

from functools import reduce # So Chris can pretend he's a CS major

import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt # For simple graphing
import seaborn as sns # For sns.set_style("whitegrid"), I guess?

## This sets the plot style
## to have a grid on a white background
sns.set_style("whitegrid")

C:\Users\chris\Anaconda3\envs\Boot_Camp\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Prepare data

In [3]:
# Read "well_formatted_json_files.txt"

data_files_to_read_path = Path('well_formatted_json_files.txt')
with open(data_files_to_read_path) as file:
    lines = file.read().splitlines()
    # Remove commented lines and blank lines:
    lines = [li for li in lines if ((li[0] != '#') and ( li.strip()))]
    print(len(lines))

10


In [4]:
# Read in json and tsv files

data_path = Path('data')

l = list()
for li in lines:
    # Read into dataframes
    
    json_path = data_path / Path(li + '_tweets.json')
    with open(json_path) as file:
        very_raw_json_contents = json.load(file)
    raw_json_contents = [x['user'] for x in very_raw_json_contents]
    json_contents = pd.DataFrame(raw_json_contents)
    tsv_path = data_path / Path(li + '.tsv')
    tsv_contents = pd.read_csv(tsv_path, sep='\t')
    tsv_contents.columns = ['user_id','species']
    
    # Set indexes to user ids and remove duplicate indices
        
    json_contents = json_contents.set_index('id')
    tsv_contents = tsv_contents.set_index('user_id')
    json_contents = json_contents.loc[~json_contents.index.duplicated(keep='last')]
    tsv_contents = tsv_contents.loc[~tsv_contents.index.duplicated(keep='last')]
    
    # Merge
    
    merged_data = pd.concat([json_contents, tsv_contents], join='inner', axis = 1)
    merged_data = merged_data.assign(source=li)
    
    l.append(merged_data)


cumulative_raw_data = reduce(lambda x, y : pd.concat([x,y]), l)

In [5]:
columns_to_keep = ['has_extended_profile','profile_use_background_image','profile_background_image_url_https','verified','profile_text_color','profile_sidebar_fill_color','followers_count','profile_sidebar_border_color','profile_background_color','listed_count','utc_offset','statuses_count','profile_link_color','geo_enabled','profile_background_image_url','lang','profile_background_tile','favourites_count','url','created_at','time_zone','default_profile','friends_count']

target = cumulative_raw_data['species'] == 'bot'
# old code: target = np.asarray([cumulative_raw_data['species'] == 'bot']).reshape(-1,1)
source = cumulative_raw_data['source']
cut_raw_data = cumulative_raw_data[columns_to_keep]

cut_raw_data.sample(3)

,has_extended_profile,profile_use_background_image,profile_background_image_url_https,verified,profile_text_color,profile_sidebar_fill_color,followers_count,profile_sidebar_border_color,profile_background_color,listed_count,...,geo_enabled,profile_background_image_url,lang,profile_background_tile,favourites_count,url,created_at,time_zone,default_profile,friends_count
852512004,False,True,https://abs.twimg.com/images/themes/theme1/bg.png,False,333333,DDEEF6,17,C0DEED,C0DEED,0,...,True,http://abs.twimg.com/images/themes/theme1/bg.png,pl,False,117,None,Sat Sep 29 08:52:13 +0000 2012,Brussels,True,0
124735548,False,True,https://abs.twimg.com/images/themes/theme1/bg.png,False,333333,DDEEF6,118,C0DEED,C0DEED,5,...,False,http://abs.twimg.com/images/themes/theme1/bg.png,en,False,1984,None,Sat Mar 20 11:32:59 +0000 2010,None,True,352
120557765,False,True,https://abs.twimg.com/images/themes/theme9/bg.gif,True,666666,252429,2872283,181A1E,1A1B1F,8221,...,False,http://abs.twimg.com/images/themes/theme9/bg.gif,en,False,376,https://t.co/rJixLu4cJy,Sat Mar 06 20:33:05 +0000 2010,Alaska,False,1816


In [36]:
# Keep numerical and boolean attributes. Don't keep categorical attributes that haven't been encoded yet.

numerical_cols = ['favourites_count','statuses_count','friends_count','followers_count','listed_count']
boolean_cols = ['profile_background_tile','default_profile','geo_enabled','verified','has_extended_profile','profile_use_background_image']

assert set(numerical_cols) <= set(cut_raw_data.columns)
assert set(boolean_cols) <= set(cut_raw_data.columns)

cleaned_data = cut_raw_data[numerical_cols + boolean_cols]

display(cleaned_data.sample(3, random_state=855))
print(cleaned_data.shape)

,favourites_count,statuses_count,friends_count,followers_count,listed_count,profile_background_tile,default_profile,geo_enabled,verified,has_extended_profile,profile_use_background_image
1870916851,132,30,0,1,0,False,True,False,False,False,True
340052871,2497,7452,183,199,0,True,False,True,False,False,True
156031870,6635,19851,146,259,4,True,False,True,False,True,True


(44595, 11)


NUMERICAL:

* favourites_count
* statuses_count
* friends_count
* followers_count
* listed_count

BOOLEAN:

* profile_background_tile
* default_profile [boolean]
* geo_enabled
* verified
* has_extended_profile
* profile_use_background_image

CATEGORICAL (NOT YET ADDED):

* profile_background_image_url_https [consider categorical: equal to theme1, theme14, NaN, or other]
* url (NaN or other)
* lang [categorical: en or other (maybe NaN, es, pt, it, too)]
* profile_link_color [categorical: 1DA1F2 or other]
* profile_background_image_url [categorical: theme1, NaN, or other]
* profile_sidebar_border_color [consider categorical: either equal to C0DEED, FFFFFF, 000000, or other]
* profile_background_color [categorical: equal to C0DEED, F5F8FA, 000000, or other]
* profile_text_color [consider categorical: either equal to 333333 or not]
* profile_sidebar_fill_color [consider categorical: either equal to DDEEF6 or not]

WEIRD/OTHER (NOT GOING TO BE ADDED)

* utc_offset [categorical? UTC offset measured in seconds]
* created at [look at day or week, or at time of day]
* time_zone [categorical: NaN or other]

## Train model

The train-test split stratifies by target right now, and not by source. We should stratify by both. (Note to Chris: Think of stratifying by a source-target ordered pair.)

In [41]:
# This is an array of ordered pairs. Each pair combines the bot/human value of
# an account with the source dataset from which the account came. We need this
# so that train_test_split can stratify by bot/human value AND data source.

# This codeblock takes ten seconds to run on my PC.

stratify_guide = np.asarray([(target[acct],source[acct]) for acct in cleaned_data.index]).reshape(-1,1)

In [42]:
# Train test split

samp_size = 1000
X_train,X_test,y_train,y_test = train_test_split(cleaned_data[['favourites_count']],
                                                target,
                                                train_size=samp_size,
                                                shuffle=True,
                                                stratify=target,
                                                random_state=855)

I tried hyperparameter tuning of `rf_model` but I couldn't really get the accuracy to budge. Adding more attributes to the training data would probably help.

In [43]:
rf_model = RandomForestClassifier(n_estimators = 100,
                                  max_features = 'auto',
                                  criterion='entropy')
rf_model.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

## Test model

y is True for bots, False for humans

In [44]:
pred = rf_model.predict(X_test)
print('overall accuracy:', np.sum(pred == y_test)/len(y_test))
print()

for src in set(source.values):
    print(src)
    print('accuracy:', np.sum( (pred == y_test) & (source == src) )/source.value_counts()[src])
    print()

overall accuracy: 0.6906296593646061

vendor-purchased-2019
accuracy: 0.6151012891344383

verified-2019
accuracy: 0.6379657603222558

botwiki-2019
accuracy: 0.7474892395982783

cresci-rtbust-2019
accuracy: 0.5708092485549133

celebrity-2019
accuracy: 0.4725367584924793

cresci-stock-2018
accuracy: 0.575668549905838

pronbots-2019
accuracy: 0.8492254348190817

botometer-feedback-2019
accuracy: 0.5667311411992263

political-bots-2019
accuracy: 0.7540983606557377

gilani-2017
accuracy: 0.6121627064035441

